# Lab Solution - Adversarial Search #

This notebook serves as the starter code and lab description for the Lab covering **Chapter 5 - Adversarial Search** from the book *Artificial Intelligence: A Modern Approach.*

In [1]:
from starter import *

# This function is placed here to help you read through the source code of different classes, 
#  and debug what has been loaded into jupyter, 
#  make sure all the function calls to `psource` are commented in your submission
def psource(*functions):
    """Print the source code for the given function(s)."""
    from inspect import getsource
    source_code = '\n\n'.join(getsource(fn) for fn in functions)
    try:
        from pygments.formatters import HtmlFormatter
        from pygments.lexers import PythonLexer
        from pygments import highlight
        from IPython.display import HTML

        display(HTML(highlight(source_code, PythonLexer(), HtmlFormatter(full=True))))

    except ImportError:
        print(source_code)

## OVERVIEW
We exercise adverserial search in terms of the game tic-tac-toe, a very simple game but complex enough to help us practice what we learned in the lecture.

We start with defining the abstract class `Game`, for turn-taking *n*-player games. A game is similar to a problem, but it has a terminal test instead of a goal test, and a utility for each terminal state. To create a game, subclass `Game` and implement `actions`, `result`, `is_terminal`, and `utility`. You will also need to set the `.initial` attribute to the initial state; this can be done in the constructor.

We rely on, but do not define yet, the concept of a `state` of the game; we'll see later how individual games define states. For now, all we require is that a state has a `state.to_move` attribute, which gives the name of the player whose turn it is. ("Name" will be something like `'X'` or `'O'` for tic-tac-toe.) 

We also define `play_game`, which takes a game and a dictionary of  `{player_name: strategy_function}` pairs, and plays out the game, on each turn checking `state.to_move` to see whose turn it is, and then getting the strategy function for that player and applying it to the game and the state to get a move.

In [2]:
# psource(Game)
# psource(play_game)

# Tic-Tac-Toe and Board

We have the notion of an abstract game, based on it, we define a real game; a simple one, `TicTacToe`. Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(2, 2)` is the bottom right (on a board of size `height=width=3`). You need `k` squares in a row to win.

States in tic-tac-toe (and other games) will be represented as a `Board`, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes: 
- `.to_move` to name the player whose move it is ('X' plays first against 'O'); 
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- possibly other attributes, as specified by keywords. 

A `Board` has the player to move, a cached utility value, and a dict of `{(x, y): player}` entries, where player is `X` or `O`. As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `off` otherwise. The class has a `__hash__` method, so instances can be stored in hash tables.

In [3]:
# psource(TicTacToe)
# psource(Board)

# Players

We need an interface for players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below:

In [4]:
# psource(random_player)
# psource(player)

# Minimax-Based Game Search Algorithms

Now, we will define several game search algorithms. Each takes two inputs, the game we are playing and the current state of the game, and returns a a `(value, move)` pair, where `value` is the utility that the algorithm computes for the player whose turn it is to move, and `move` is the move itself.

First we define `minimax_search`, which exhaustively searches the game tree to find an optimal move (assuming both players play optimally), and `alphabeta_search`, which does the same computation, but prunes parts of the tree that could not possibly have an affect on the optimnal move.  

In [5]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    @cache
    def max_value(state):
        # TODO return the game utility if game is in a terminal state.
        # TODO in all possible game actions choose the action that is the best one and return the action along its value as a (value, action) pair.
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move
    @cache
    def min_value(state):
        # TODO return the game utility if game is in a terminal state.
        # TODO in all possible game actions choose the action that is the best one and return the action along its value as a (value, action) pair.
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

infinity = math.inf

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move
    
    @cache
    def max_value(state, alpha, beta):
        # TODO return the game utility if game is in a terminal state.
        # TODO in all possible game actions choose the action that is the best one and return the action along its value as a (value, action) pair.
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move
    
    @cache
    def min_value(state, alpha, beta):
        # TODO return the game utility if game is in a terminal state.
        # TODO in all possible game actions choose the action that is the best one and return the action along its value as a (value, action) pair.
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

# Playing a Game

We're ready to play a game. I'll set up a match between a `random_player` (who chooses randomly from the legal moves) and a `player(alphabeta_search)` (who makes the optimal alpha-beta move; practical for tic-tac-toe, but not for large games). The `player(alphabeta_search)` will never lose, but if `random_player` is lucky, it will be a tie.

In [6]:
play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility

Player X move: (0, 2)
. . .
. . .
X . .

Player O move: (1, 1)
. . .
. O .
X . .

Player X move: (2, 1)
. . .
. O X
X . .

Player O move: (1, 2)
. . .
. O X
X O .

Player X move: (2, 0)
. . X
. O X
X O .

Player O move: (1, 0)
. O X
. O X
X O .



-1

The alpha-beta player will never lose, but sometimes the random player can stumble into a draw. When two optimal (alpha-beta or minimax) players compete, it will always be a draw:

In [7]:
play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (2, 1)
. . .
X . O
. . .

Player X move: (1, 2)
. . .
X . O
. X .

Player O move: (0, 0)
O . .
X . O
. X .

Player X move: (1, 1)
O . .
X X O
. X .

Player O move: (1, 0)
O O .
X X O
. X .

Player X move: (2, 0)
O O X
X X O
. X .

Player O move: (0, 2)
O O X
X X O
O X .

Player X move: (2, 2)
O O X
X X O
O X X



0

# Connect Four
[`ConnectFour`](https://connect-4.org/en) is a variant of tic-tac-toe, played on a larger (7 x 6) board, and with the restriction that in any column you can only play in the lowest empty square (first empty square starting from the bottom) in the column.

In [8]:
# psource(ConnectFour)

Try running the same code you just implemented for `ConnectFour` and report if there is any problem (you may kill this process once you found a problem). 

In [9]:
# play_game(ConnectFour(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility

Try modifying your `alphabeta_search` to alleviate the problem and report the changes in result.

In [10]:
def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def h_alphabeta_search(game, state, cutoff=cutoff_depth(6), h=lambda s, p: 0):
    """Search game to determine best action; use alpha-beta pruning and search all the way to the leaves."""

    player = state.to_move

    @cache
    def max_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    @cache
    def min_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth + 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity, 0)

In [11]:
play_game(ConnectFour(), {'X':player(h_alphabeta_search), 'O':random_player}, verbose=True).utility

Player X move: (5, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .

Player O move: (3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . X .

Player X move: (3, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . . O . X .

Player O move: (0, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
O . . O . X .

Player X move: (0, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . X . . .
O . . O . X .

Player O move: (3, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . . X . . .
O . . O . X .

Player X move: (6, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . . X . . .
O . . O . X X

Player O move: (0, 3)
. . . . . . .
. . . . . . .
. . . . . . .
O . . O . . .
X . . X . . .
O . . O . X X

Player X move: (1, 5)
. . . . . . .
. . . . . . .
. . . . . . .
O . . O . . .
X . . X . . .
O X . O . X X

Player O move: (6, 4)
. . . . . . .
.

1